In [17]:
import glob

exclude = ['./CLAUDE.md','./progress-tracker.md','./project-description.md']
list_of_files = glob.glob('meridian-islands/**/*.md') 
filtered_list = []
for a in list_of_files:
    if a not in exclude:
        filtered_list.append(a)

filtered_list = sorted(filtered_list)

current_list = filtered_list[0]
print(current_list)

markdown_string = " "

with open(current_list, "r", encoding="utf-8") as f:
    converted_text = f.read() 

with open("first.txt", "w", encoding="utf-8") as text_file:
    text_file.write(converted_text)

meridian-islands/01-foundation/world-bible.md


In [ ]:
from dataclasses import dataclass

class Chunk_class:
    def __init__(self,converted_text):
        self.converted_text = converted_text
        self.chunks = []

    @dataclass
    class Chunk_attributes:
            text: str
            chunk_id: int
            start_pos: int
            end_pos: int
            char_count: int
            origin_file: str
                    

    def create_chunk(self, converted_text):
        chunks = []
        i=0

        for start in range(0,len(converted_text),200):
            end = min(start+200, len(converted_text))
            chunk = converted_text[start:end]
        
        for start in (range(0, len(converted_text), 200)):
            end = min(start+200, len(converted_text))
            chunk = converted_text[start:end]
            i += 1
            chunks.append(
                Chunk_class.Chunk_attributes(
                    text=chunk,
                    chunk_id=i+1,
                    start_pos=start,
                    end_pos=end-1,
                    char_count=len(chunk),
                    origin_file=current_list
                )

            )
        return chunks
            
a = Chunk_class(converted_text)
a.create_chunk(converted_text)


        

[Chunk_class.Chunk_attributes(text='# Meridian Islands World Bible\n*Master Reference Document - All content must align with these established facts*\n\n## Archipelago Overview\n\nThe Meridian Islands are a sovereign island nation located in', chunk_id=2, start_pos=0, end_pos=199, char_count=200, origin_file='meridian-islands/01-foundation/world-bible.md'),
 Chunk_class.Chunk_attributes(text=' the South Pacific Ocean, approximately 2,100 kilometers east of Australia and 1,800 kilometers north of New Zealand. The archipelago consists of six islands: four main inhabited islands and two small', chunk_id=3, start_pos=200, end_pos=399, char_count=200, origin_file='meridian-islands/01-foundation/world-bible.md'),
 Chunk_class.Chunk_attributes(text='er islets.\n\n### The Six Islands\n\n1. **Coral Bay Island** (385 km²) - The capital island and main tourist hub\n2. **Azure Peak Island** (298 km²) - Adventure and nature tourism center\n3. **Amber Cove Is', chunk_id=4, start_pos=400, end_pos=599